In [10]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys
from devtools import debug

[sys.path.append(str(path)) for path in [Path.cwd(), Path.cwd().parent, Path.cwd().parent/"python"] if str(path) not in sys.path]

[None, None]

In [3]:
from langchain_community.chat_models import ChatLiteLLM
from langchain_core.messages import HumanMessage

chat = ChatLiteLLM(model="gpt-3.5-turbo")

messages = [
    HumanMessage(
        content="Translate this sentence from English to French. I love programming."
    )
]
chat(messages)

ImportError: cannot import name 'ChatLiteLLM' from 'langchain_community.chat_models' (/home/tcl/.cache/pypoetry/virtualenvs/genai-training-GvhVNXsu-py3.10/lib/python3.10/site-packages/langchain_community/chat_models/__init__.py)

In [12]:
d =  {'title': 'PromptInput', 'type': 'object', 'properties': {'query': {'title': 'Query', 'type': 'string'}}}
keys = d.get("properties").keys()
len(keys)
debug (keys)
next(iter(keys))

/tmp/ipykernel_1274359/2630313233.py:4 <module>
    keys: dict_keys(['query']) (dict_keys) len=1


'query'